# เตรียม โฟเดอร์ ที่มี ไฟล์ภาพจาก imageJ thแล้ว และ มี ไฟล์ json (ทำการเข้า via vigg มาแล้ว)
และ ต้องให้ไปอ่านภาพ ที่ทำการ threshold เองมาก่อน

In [ ]:
import os
os.chdir("/content/drive/My Drive/Mask_RCNN_M2C/MaskRCNN_Image_Segmentation-master/samples/leaf_script/mask_image_from_test_input/รวมimageJ/json_format_from_via")#new_add_dataset_more/real_keep_can_all")
!ls

In [ ]:
import json

file_json = "via_region_data_before_edit.json" #'via_region_data_156.json'                # file name from via vgg
with open(file_json) as json_file:
    data = json.load(json_file)  # แปลง JSON ไฟล์ เป็น Python Dict #คืออ่านไฟล์ json ที่ได้ format จาก via vgg มาแล้วแก้ไขใหม่

length = (len(data))

dict2list_data = list(data)
get_key2acess = []

for x in range(length):
    # print(dict2list_data[x])
    get_key2acess.append(dict2list_data[x])  

len(get_key2acess) # จำนวน
# get_key2acess

In [ ]:
import os #จะเขียนไฟล์ใหม่ลงใน dir นี้ และต้องมีภาพ จาก ImageJ
# dir_ = "/content/drive/My Drive/Mask_RCNN_M2C/MaskRCNN_Image_Segmentation-master/samples/leaf_script/mask_image_from_test_input/test_overlay_imageJ/imageJ" #'/content/drive/My Drive/Colab Notebooks/ImageJ_Threshold_for_get_XY_process/threshold_all_for_get_X_Y' # real last 
# dir_ = '/content/drive/My Drive/Mask_RCNN_M2C/MaskRCNN_Image_Segmentation-master/samples/leaf_script/mask_image_from_test_input/BenchMark_2' #test if can do
dir_ = "/content/drive/My Drive/Mask_RCNN_M2C/MaskRCNN_Image_Segmentation-master/samples/leaf_script/mask_image_from_test_input/รวมimageJ/"
os.chdir(dir_)
!ls

In [ ]:
import os

file_jpg = []
for file in os.listdir(dir_):
    # print(file.split('.')[0])
    if file.endswith(".jpg"):
        # print(os.path.join(dir_, file))
        file_jpg.append(os.path.join(dir_, file))
len(file_jpg)

In [ ]:
from os import listdir
from os.path import isfile, join

all_files = [f for f in listdir(dir_) if isfile(join(dir_, f))]
len(all_files) # + 1 json file

In [ ]:
# ทำเป็นฟังชัน เป็นการกรอง noise ออก โดยเลือก area ของ contour ที่มีค่ามากกว่ size2filter ที่ต้องการ ***
def deleteNoiseAndFilled(blackAndWhiteImage, size2filter):

  #หา contour จาก mask
  cnt_, hierarchy_ = cv2.findContours(blackAndWhiteImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

  #สร้าง ภาพสีดำตาม ขนาดภาพที่ผ่านเข้ามา
  filterNoise = np.zeros(blackAndWhiteImage.shape[:2], dtype="uint8")
    
  color_black = (0,0,0)
  color_white = (255,255,255)

  for i in range(len(cnt_)): # ต้องวนอ่าน ค่าพิกัด x,y จาก จำนวนคอนทัว ทั้งหมดก่อน
    area_ = cv2.contourArea(cnt_[i]) # หาพื้นที่ ทุกคอนทัว
    # print(area_)
    if area_ > size2filter: # พื้นที่ ที่มากกว่า size2filter ที่กำหนดมา
      cnt_filter = cnt_[i] # กำหนด ค่า cnt ใหม่ โดยในที่นี้จะเป็นค่าที่อยู้ในเงื่อนไขแล้ว ***
      # print(cnt_) # คือพิกัด 
      # print(i) #จำนวน คอนทัว ทั้งหมด
      # print(area) # พื้นที่
      cv2.drawContours(image=filterNoise, contours=[cnt_filter], contourIdx=-1, color=color_white, thickness=cv2.FILLED)  
  return filterNoise

In [ ]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

        #ปกติ json ไม่รู้จักตัว array ของ python แต่เราทำการเก็บข้อมูล x,y ลงใน array จึงทำการแปลง npเป็น generic
        # generic คือ  Data type ของตัวแปรที่จะใช้ใน Class หรือ Method นั้นๆ ให้มีลักษณะ dynamic ตามการเรียกใช้งาน  ในที่นี้ json รุ้จักแค่ int array ที่เข้ามาจึงกลายเป็นตัวเลขและเขียนลงได้


In [ ]:
%%time
# all
import time
import os, os.path
import cv2 
import numpy as np 
from google.colab.patches import cv2_imshow

# simple version for working with CWD or use '.'
# print(len([name for name in os.listdir(dir_) if os.path.isfile(os.path.join(dir_, name))]))
# import random
# random.seed(123)

for file in os.listdir(dir_): # วนตามไฟล์
  if file.endswith('jpg') == True:
    for idx, key in enumerate(get_key2acess):
      # print("file"+file_name.split('.')[0])
      # print('key'+key.split('.')[0])
      if int(file.split('.')[0]) == int(key.split('.')[0]):                                                
        # print(key)     

        
        if file == '192.jpg' :                                                            # ลองรันไฟล์เดียว
      
          image_ben = cv2.imread(dir_+'/'+ file)#, 0 )
          # cv2_imshow(image_ben)

          #threshold ใช้ได้กับภาพ bw เท่านั้น เพราะการอ่านภาพมาใหม่ จะอ่านเป็น bgr 
          img_copy_gray = cv2.cvtColor(image_ben, cv2.COLOR_BGR2GRAY)
          # img_copy_gray.shape
          (thresh, blackAndWhiteImage_) = cv2.threshold(img_copy_gray, 65, 255, cv2.THRESH_BINARY)
          # cv2_imshow(blackAndWhiteImage_)

          filterForEdge = deleteNoiseAndFilled(blackAndWhiteImage=blackAndWhiteImage_,size2filter=200)
          # cv2_imshow(filterForEdge)

  # OLD EGDES FIRST
          # # filterForEdge
          # edges = cv2.Canny(filterForEdge, 100, 255)
          # # https://java2blog.com/cv2-canny-python/

          # time.sleep(1) #1

          # # find coordinates having value more than 0
          # indices = np.where(edges != [0])
          # # used the numpy.where() method to retrieve a tuple indices of two arrays where the first array contains the x-coordinates of the white points
          # #  and the second array contains the y-coordinates of the white pixels.
          # coordinates = zip(indices[0], indices[1])
  #//

        # ต้องทำการแยก roi 
          height,width = filterForEdge.shape

          x1=0
          y1=0
          x2=width
          y2=height

          half_height =y1+y2 //2 # มี 2 row
          column_current = x1+x2 // 3 # มี 3 column 

          filterForEdge_copy = filterForEdge.copy()

          all_x_coordinate = []
          all_y_coordinate = []

          time.sleep(1) #

          for row in range(0,2):
            # print(row)
            for column in range(0,3):
              # print(column)
              if row == 0 :
                # cv2_imshow(edges[y1:half_height ,x1 + (column*column_current) :column_current + (column*column_current) ])
                # filterForEdge_copy_current_roi = filterForEdge_copy[y1:half_height  ,x1 + (column*column_current) :column_current + (column*column_current) ].copy()
                # print("column",column)

                if column  == 0 : # ในที่นี้คือภาพแรก ไม่สามารถตัดมาทางซ้ายอีกได้ แต่เพิ่มไปทางขวา กับลงล่างได้  
                  # print("0")
                                                                      # 0           :1122                        # 0           : 714
                  filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) :column_current + (column*column_current) +150].copy()
                  # print("this",filterForEdge_copy_current_roi.shape)

                elif column == 1 : # คือ ภาพกลาง สามารถลบ ทั้งด้านซ้าย และขวาได้
                  # print("1")
                  filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) -150 :column_current + (column*column_current) +150].copy()
                
                elif column == 2 : # คือ ภาพซ้ายสุด สามารถลบด้านซ้าย ได้อย่างเดียว
                  # print("2")
                  filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) -150 :column_current + (column*column_current) ].copy()
                                                                                                            
              elif row == 1: # ภาพด้านล่าง # old
                # cv2_imshow(edges[half_height: ,x1 + (column*column_current) :column_current + (column*column_current) ])
                # filterForEdge_copy_current_roi = filterForEdge_copy[half_height:     ,x1 + (column*column_current)  :column_current + (column*column_current) ].copy() # old 
                
                if column == 0 : # คือ ภาพซ้ายล่างสุด สามารถเพิ่มได้แต่ทางขวา ได้อย่างเดียว กับเพิ่มด้านบนอีก โดยการลบ
                  # print("0")
                  filterForEdge_copy_current_roi = filterForEdge_copy[half_height -150 :               ,x1 + (column*column_current)  :column_current + (column*column_current) +150  ].copy()
                
                elif column == 1 : # คือ ภาพซ้ายกลาง สามารถลบ ทั้งด้านซ้าย และขวาได้
                  # print("1")
                  filterForEdge_copy_current_roi = filterForEdge_copy[half_height -150 :               ,x1 + (column*column_current) -150 :column_current + (column*column_current) +150  ].copy()
                
                elif column == 2 : # คือ ภาพซ้ายล่าง ลบแค่ด้านซ้าย
                  # print("2")
                  filterForEdge_copy_current_roi = filterForEdge_copy[half_height -150 :               ,x1 + (column*column_current) -150 :column_current + (column*column_current) ].copy()


              left_big = np.zeros((filterForEdge_copy_current_roi.shape[:2]), dtype="uint8") # new เอาค่าที่ได้มาใหม่ และ +- ด้วยค่าที่ลอง เพื่อให้ภาพ ครอบคลุม edges

            # อยู่ในลูปเพราะ ต้องการสร้างใหม่ตลอด
              # left_big = np.zeros((half_height,column_current), dtype="uint8") #เท่ากับที่เราจะแบ่ง
              # print(left_big.shape)
              
            #ใหม่ ให้ get x,y จาก contour ในภาพ ที่มีค่า area มากสุด (เพราะอาจมีกรณรที่มีภาพอื่นเข้ามาใน roi ด้วย)
              cnt_edge_fill_current_roi, hierarchy_fill_current_roi = cv2.findContours(filterForEdge_copy_current_roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
              
              # วนก่อนเพื่อเก็บค่าทั้งหมด วน1 หาค่าพื้นที่ทั้งหมด
              area_in_each_roi_all = []
              for i in range(len(cnt_edge_fill_current_roi)): # ต้องวนอ่าน ค่าพิกัด x,y จาก จำนวนคอนทัว ทั้งหมดก่อน
                area_in_each_roi = cv2.contourArea(cnt_edge_fill_current_roi[i]) # หาพื้นที่ ทุกคอนทัว
                area_in_each_roi_all.append(area_in_each_roi)

              # วน 2 หาพื้นที่นั้นจากค่า ในlist เพื่อนำค่านั้นมาใช้
              for i in range(len(cnt_edge_fill_current_roi)): # ต้องวนอ่าน ค่าพิกัด x,y จาก จำนวนคอนทัว ทั้งหมดก่อน
                area_in_each_roi = cv2.contourArea(cnt_edge_fill_current_roi[i]) # หาพื้นที่ ทุกคอนทัว
                # print(area_in_each_roi)

                if area_in_each_roi == np.max(area_in_each_roi_all): # พื้นที่ ที่มากกว่า จากค่า พื้นที่ที่มากที่สุด ในการสรรอบแรก
              
                  cnt_big = cnt_edge_fill_current_roi[i] # กำหนด ค่า cnt ใหม่ โดยในที่นี้จะเป็นค่าที่อยู้ในเงื่อนไขแล้ว ***
                  # print(cnt_edge_fill_current_roi) # คือพิกัด 
                  # print(i) #จำนวน คอนทัว ทั้งหมด
                  # print(area_in_each_roi) # พื้นที่
                  cv2.drawContours(image=left_big, contours=[cnt_big], contourIdx=-1, color=(255,255,255), thickness=cv2.FILLED) 
              
                # ได้ภาพที่พื้นที่มากที่สุดมาแล้ว
                  # cv2_imshow(left_big)
                  # print(left_big.shape)
            
                # ค่อยมาหา edges 
                  left_big_edges = cv2.Canny(left_big, 100, 255)
                  cv2_imshow(left_big_edges)                                            # ดูได้
                  # https://java2blog.com/cv2-canny-python/
                  
                  time.sleep(1) #1

                  indices_current_roi_edges_left_big = np.where(left_big_edges != [0])
                  
                  # print("c",(max(indices_current_roi_edges_left_big[1])+column*column_current))
                  # print("r",(max(indices_current_roi_edges_left_big[0])+row*half_height))
              
              # ครั้งแรก ผิด
              ## all_x_coordinate.append(indices_current[0] + (column*column_current))
              ## all_y_coordinate.append(indices_current[1] +  (row*half_height)) 
              
              ##try new #ถูกต้อง แต่ค่าเยอะเกิน # และจะผิดที่ตรงขยายไม่ได้ คือถ้ากรอบ roi ไม่ครบ มันจะตีพลาด แต่ edges ถูก
              # all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current))
              # all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)) 
              # เพราะว่าเราตัดออก มันเลยไม่ได้เป็น (เพราะเลือก roi ) coordinate จริง ตาม edges จึงทำการบวก ตามที่ได้ตัดไป จึงทำให้รู้ค่าแล้ว
              

  # ลอง จากที่ปริ้นดู ก็ลองเอาไปเทียบ ถ้าไม่ถูกก็กลับมาแก้ ได้พลาดด้านล่าง
              # all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current) -150)
              # all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height) -150 ) 

  # สรุป ดูจากภาพด้านล่างก็รูปแล้วว่าควรแก้ค่าไหนที่เท่าไหร่

              # ทำการขยายช่วง ต้องบวกค่าที่ขยายเพิ่ม ตามเงื่อนไข แต่เรารู้เลยก็ดูจากภาพ
              if row == 0:  #ครึ่งบน 
                if column  == 0 : #ภาพแรก
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current)       ) 
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)            ) #$ ไม่ต้องมันถูกอยุ่แล้ว เพราะในภาพมันขยับไป ทาวซ้าย กะขึ้นบน เพราะลบ ขวา กะล่าง

                  # filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) :column_current + (column*column_current) +150].copy()

                elif column == 1 : # ภาพกลาง
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current) - 150 ) # จากที่ดูความตรงคือ column ตรงอยุ่แล้ว ก็ปล่อยไว้
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)            ) # ในภาพมันสูง ก็เพราะเราลบ -150 ดังนั้นก็ไม่ต้องลบ 

                  # filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) -150 :column_current + (column*column_current) +150].copy()
                
                elif column == 2 :  #ภาพขวาสุด
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current) - 150 ) # เหมือนกันกับอันกลาง
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)            ) 

              else : #row = 1 คือครึ่งล่าง
                if column  == 0 : #ภาพแรก
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current)       ) #/ ไม่ถูกเพราะมันเลื่อนไปทางซ้าย ก็จากที่ลบ ก็ไม่ต้องลบ
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)       -150 )  # ลบด้วย เพื่อไม่ให้ภาพมันเกิน

                  # filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) :column_current + (column*column_current) +150].copy()

                elif column == 1 : # ภาพกลาง
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current) - 150 ) 
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)      - 150 ) # ในภาพมันถูก คือคงค่าที่ลองไว้ 

                  # filterForEdge_copy_current_roi = filterForEdge_copy[y1              :half_height +150 ,x1 + (column*column_current) -150 :column_current + (column*column_current) +150].copy()
                
                elif column == 2 :  #ภาพขวาสุด
                  all_x_coordinate.append(indices_current_roi_edges_left_big[1] + (column*column_current) - 150 ) # เหมือนกันกับอันกลางล่าง
                  all_y_coordinate.append(indices_current_roi_edges_left_big[0] +  (row*half_height)      - 150 ) 
              # เขียนเสร็จ 1 ตัว

          #เขียนเสร็จ 6 ตัวแล้ว
          print(file)
          #เอาค่าที่เก็บ ใน all_?_coordinate มาเขียน โดยวนอ่านทีละ all_?_coordinate[index]
          # time.sleep(1) #

          # --
          regions = 0 
                    #== เหมือนหาร เอาค่าทุกๆ 5 ตัว ที่ผ่านไป ในค่าของ all_x_coordinate

          for regions in range(0,6):
            # print("before x",len(all_x_coordinate[regions]))                           #debug
            # print("before y",len(all_y_coordinate[regions]))                           #debug

            # #== เหมือนหาร เอาค่าทุกๆ 5 ตัว ที่ผ่านไป ในค่าของ all_x_coordinate
            reduce_each_coordinate_value_x = [] #เอาตัวนี้ไปเขียนลง ใน x 
            reduce_each_coordinate_value_y = [] #เอาตัวนี้ไปเขียนลง ใน y
            for index,keep in enumerate(all_x_coordinate[regions]):
              if int(index) % 8 == 0:
                # print(keep)
                # print(all_x_coordinate[0][index])
                reduce_each_coordinate_value_x.append(keep)
            # print("after X",len(reduce_each_coordinate_value_x))                        #debug
            
            for index_,keep_ in enumerate(all_y_coordinate[regions]):
              if int(index_) % 8 == 0:
                # print(keep_)
                # print(all_x_coordinate[0][index])
                reduce_each_coordinate_value_y.append(keep_)
            # print("after Y",len(reduce_each_coordinate_value_y))                        #debug
          
            if int(file.split('.')[0]) <= 100 :
              class_ = "1" # green 
            else :
              class_ = "2" # red

            data[key]["regions"][regions] = {}
            # for regions in range(0,6):
            # print(regions)                                                              #debug
            data[key]["regions"][regions] = {"shape_attributes":{"name":"polygon","all_points_x":list(reduce_each_coordinate_value_x) , "all_points_y": list(reduce_each_coordinate_value_y)}, "region_attributes":{"class":"{}".format(class_)} }
          with open('via_region_data_edit_real_reduced_all.json', 'w') as json_file:
            json.dump(data, json_file , default=np_encoder)  # เขียน Python Dict ลงในไฟล์ "?"
            #--
          



          # regions = 0 # กำหนดครั้งแรกครั้งเดียว #same
          
          # if int(file.split('.')[0]) <= 100 :
          #   class_ = "1" # green 
          # else :
          #   class_ = "2" # red

          # # regions = 0 # กำหนดค่าครั้งแรก วนทุกครั้งให้เริ่มใหม่ที่ regions = 0 ก่อนจะเข้าไปวนสร้างใหม่ #same
          # data[key]["regions"][regions] = {}
          # for regions in range(0,6):
          #   # print(regions)                                  #
          #   # print(list(all_x_coordinate[regions]))            ##debug
          #   data[key]["regions"][regions] = {"shape_attributes":{"name":"polygon","all_points_x":list(all_x_coordinate[regions]) , "all_points_y": list(all_y_coordinate[regions])}, "region_attributes":{"class":"{}".format(class_)} }
          # with open('via_region_data_edit_.json', 'w') as json_file:
          #   json.dump(data, json_file , default=np_encoder)  # เขียน Python Dict ลงในไฟล์ "?"
